## Audio experiments

In [132]:
#>flags -d:blas=cblas -p:~/code/misc/jnim

# Original Python code:
# srate = 11000
# dur = 1000
# rr = np.arange(dur*2)
# wave = np.sin(2*np.pi*400*rr/srate) * np.sin(0.5*np.pi*rr/dur)
# plt.plot(rr, wave)
# Audio(wave, rate=srate, autoplay=True)
    
import arraymancer, pixie, sequtils, std/[strutils, math, base64, strformat], wavfile, vorbis, jupyternimpkg/display

proc pixieHTML(img: Image): string =
    return fmt"""<img src="data:image/png;base64, {img.encodeImage(PngFormat).encode}" />"""

proc plot1D(img: Image, arr: openArray[float], scalar: float = 1.0): Image =
    var path: seq[string] = @[]
    let r = float(img.height)/2.0
    
    # For long arrays, draw <wmul> overlapping
    let wmul = 8
    if arr.len < img.width * wmul:
        for i in 0..<arr.len:
            path.add(fmt"{int(float(i)*float(img.width)/float(arr.len))} {int(arr[i]*scalar*r + r)} ")
    else:
        for i in 0..<img.width * wmul:
            let arr_loc = int(float(i)*float(arr.len)/float(img.width)) div wmul
            path.add(fmt"{i div wmul} {int(arr[arr_loc]*scalar*r + r)} ")
        
    img.strokePath(fmt"M 0 {int(r)} L " & path.join(""), rgba(0, 0, 128, 255), strokeWidth = 1)
    return img

proc basicSine(srate: int): Tensor[system.float] =
    let srate = float(srate)
    let dur = 1000
    let rr_dur = dur * 2
    let rr = toSeq(0..<rr_dur).mapIt(it.float).toTensor()
    return sin(rr*(2.0*PI*440.0/srate)) *. sin(rr*(0.5*PI/float(dur)))


let wsample = loadWav("sample.wav")
html wsample.toHTML(false)
html pixieHTML(newImage(500,100).plot1D(wsample.toFloat, scalar=4.0))

#let srate = 11025
#let wave = basicSine(srate).toSeq
#html Audio(wave, srate, false)
#html pixieHTML(newImage(400, 100).plot1D(wave))

#let vsample = loadVorbis("sample.ogg")
#html vsample.toHTML(false)
#html pixieHTML(newImage(500,100).plot1D(vsample.toFloat))

Your browser does not support the audio element.

<img src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAfQAAABkCAYAAABwx8J9AAA4KklEQVR4Ae2dCbQtR1nv/3VuEhKTgIEECCAYkATCFBLIeHN73xCjiAg8RlFE4Yk8RQVEJuVeciIRJ3y+5VL0KT7kOSA+VEQCGbjZlxAikBBAkkBIgAgkyBgghAycfutX1d/etft09+599nB6n1u/ters6hq++mro+rqrq/tsUyKRSCQSiaUnGfREIpFIJLYAyaB3ll2vkbJM6u/trIqJRCKR6AzJoHcWjDkkg55IJBKJ8SSD3lmSQU8kEolEe5JB7yzJoCcSiUSiPcmgd5Zk0LcW7InY2Zd2rKV9EYlEYh4kg95ZkkHfWtCfbqeU70kGPZFIzINk0DsLBgDSpritAf2ZDHoikZgfyaB3FgwAJIO+NaA/k0FPJBLzIxn0zoIBgGTQtwb0ZzLoiURifiSD3lkwAJAM+taA/kwGPZFIzI9k0DsLBgCSQd8a0J8YdFza6Z5IJGZPMuidBQMAyaBvDehPjDmkne6JRGL2JIPeWTAAkAz61oD+TAY9kUjMj2TQOwsGAJJB3xrQn8tm0HevStn29N58IrEcJIPeWTAAkAz6DNHmkWWS2xnKz/dI/b3B32V6j5d0+3LomkgktqUm6CoYAIgn/l2vkXpnSP2LiUksE/Tnshn07Cck3ZIMeiKxHCSD3lkwABAbdB92kNS/iJjEMkHfddWgc6GIfvFYg95TJd28dQ16Xb3bMm3+fY3UXvMmGfTOwgQLTLK7Xy9lp3AkuUOk/nuCf9FUnZC7zpGy7Vt30p8V9GdXDTq6AWMtJnumpK9s3b6l3iur0o61jdWR/FBut0Q1qb3mTTLonSUe/NlPSvpGUNXdTeq/K/gXTayT0XuKpO9sbEJcFLtXN/+ig7ZbOoP+U5Ju6nbfToP1SbynYfdrpez0dnUmP5hBr7rgTQwpt1di1iSD3lniwZ/9vKQvBFXd4VL/HcG/aGKdjOwZkr7ebgLcLHod2NxF2y2bQe/9rKTPd7tvp8H6JDbovbMk3dmuzuQHM+jl48QoqX3mTTLonSUe/L1flXRtUNXdU+r/U/AvGtOJXxwTWe85kr7UbgLcLHpP2vzNXbTX0hl0LiQ/0+2+3SjcTbPcDrFBz35U0m3t6kyfIsOW7DkGzgvkc4w/EaA9gF/cMrTNcvVjMuidhQEPDPrsVZKu5EhaefHGn/lNi+lkoFvvv0u6od0EuFlkbO76xubqSNstnUF/YbiQpJ83SlcnxLr+6D2x/SMkk2EXBBwD7RX7EwFrE4N26jqm8zLoqmTQu0s8kHq/JenSoCtGwSaQRWM6AXcmmZPyY8OkTxyuKwM/NiS9Z23+5i7ahr6Dzeq/OtANyn3Xe5Gkqye7EIrbHepkbzboVdUffjXnW+3qbDKsPzkG2jH2JwLWJgbtVB4vXcN0Rtfuk+7QO4sNpOM/JB24Kj9pAJOQTSBVzPMEMZ0APfQByT1S0icICXRl4Juu6JP9tKQb203S8wJ9fJsp9CV6dQV0g7JO2UskfWyydivLKh93BfSq6o/efwsbUMttUYXJsPORYyBv7J+EeZ6/m421iTFpO02yYXFWmM7ouhnsep2Undb2HEwGvbPYQDrwWsm9JBgBYBKyCaQKy1c3AP2E8cNSf09V7mZMNng9PiS54yVdQUigqtzN2GVuuqJP7+ck/Wfbk2I+oA9tBk39txmgG9BWBuNk5Wlh3MXh4yjLKh93BfSq6o/saZK+2m6smAzrT46B9or9k7DRfMuA1c2YtJ3iDYuLmlNMZ3Stg3OFdE1pNkr2dKntilFacu8wDBDIvyitvCBMrMAkZBNIFZavbnD5+EOl/vlVuZvxeQu8HpdLeqz8nbpRVe5m7DI3XdHHvyVwfbtJel6gD20G/NpGqi6AbkBbGYShZ9NYq4J8YLLKx10BvagfxHXMeDzzX+3Gismw/BwDdY/9k7DRfMuA1c2YtJ16T5BWXh32ELmDF/NZYtMZXcvsOju84mhUpZkWv+n4y+3GYzLoHcYG0so3JPfTzQadK0SWCvvv1iBf3eDycu8u9f9t8sr7vAVMhvmVkjtZUr8IxFvxjfJFf0KUj92s7A5thj7ZL0n6ZNuTYj7QdrSZYUagC3jdii8QMpY4BvSdVE/LS7tD+bgroBf1g7iOvQkez5gM8uNnXwl+6s4x4J+EjeZbBqxuBm1jYfjHwYZFd2I4r91hi3lzpUm/rHgd1mhTh0nxNyOtNx2nJffOYgPJ3Sm5p4RBDExCuGzbcNmctCuvCleuRtXgYrJm0vEbnTbw6psvZzX6JyPXSSuPCMfoZJNZmYznkqVPiKIL8qr0nBY+duNOCm2GfP/a3yeSQa+DfljZNTp+oKlP60AW0O70sY2XSVckyIss5MwaZKOXEY/b3s+0f/ce/ayNwPzoTBzgn4SN5lsGrG4GbWNh+OkXjvFX0Xuy5E4I57WOaL/XYRrQB6p0yoo3IoyqNDCuXk30fkXSp9rOXcmgdxYbSDpAcmcVg5hPvxb/sUsflPoXEBLuyglnYjKqBhcyfbrrpP4/KAy0H5H677VczVj+ATdJ7oGDI19+Zbl8fKb0XBJZUJV+WijPHTfUJ/u18NrfPMpqC/Wl7Q38kxq5eWG6xeMH0JGwSdoNWUAe/MiAaeTMmlgviHXL+JjO59pNoCaH/EBd8VvdAX8VdZM8MqEu3zJjdTOoo4WV/VWwv8E9KpzX7sj5vrli/WOgX5mMr2R+cxhalQbiOk5K79cn2ZiaDHpnsUGg/Yd3wEwYRv4RqX9eOCItcUwmBoPLvgGPHyydX8L5v+FCQHx5rsWnZBng8V2N5xbJ3cv7PJRPWeXdqD0+IVr6+Ay6AOlnDc+d3LHhxEd+75WSPhwmaepB2YQvEsqkj2KsvTYb0w19YtCXsEnaClnAbzxexskp9wv5YZKy20A5sV4Q65bxXYWW+y3Q0doIzI/OxAH+Kuri68KXGdrc6hVD21h42W9YXuJ5/dQ9PJzXuv9831wxvQzKN0wnP/d9zULD/FKFyYpltKW3W9K/j8q28tfLSwa9c1hnxTBRrOPjw0/AMmBIw8Rk0Nn2DXj8YOn8K1x/VRj0+0n9txPbzCBvDI8D7joMoHzKYhc9cfiBZcyVXwhLuhaGPLDjWcLHbtyDwomP/Ow3w8Y9/D3+JWjLL4HNEupLH8VYezVh4wHd54Xphj4x6EtYm7JNzxjyG1Vydp8rZaeGyQodwMoqH88K5MZ6Qaxb9gvjv6uw+/el7CSJfQfu9DDOALkmi3LA6lOGeC4sWKXBjyMtv4B/atQNsmy9Hiur0o61YXh/rwbp8BuWl7DesyX3UCnfI7kHSvpP+fB5YOUalG8Qh/7uUUEX4Jj6xOlg12vk49C5HDcWJ/XOldQfrSflw3p52wheTnb/vpSdNFrRrYB1VozbGR8F8muk/tuDnzykYdCAO1jqXyhlPy/pC8M2snTuAdHJ9ENS/2/J1YzlHeEukttvGEL5DLLs8ZJuG5bbe144AS1+qkHeguwXJXc/+TbhJHNnSHpfmDBWfiOchOi5SKrajz7L9pf6763XhHwwL32tL4D+iUE/wtAB16QD8WXIbyCH/JRHWvzZMyR9M4wTwpgUe6+NxiZz2V7+zg7KifUCjhknlMXY0TWEBggrkz1fWnmR5E4PMdQNkIOfPJQD+KsgnvTm2uZbRqhrGepNncEdJPUvCucnxG1meQnLfkZyDw7nr46RdH0YO/PAyjUoH3adI63sxhcY1GFn0MvSAWOdMQ2ki+Pa8Iq7Sfu/er1c0229vCU26NlzJX1lfh26WVhnxTBYyuTXSv1/CKHkIQ2DxnNfqf8Oqferkq4NJ0r2w5K+F4wcDNI+XOq/iZBmrIyY2JgDMhlkvSdIK6+Sn5yzQ6X8KMn9wHBgmizS48eRbxb4k+g5Q0l5X1o5M5QN1k6zKq8O9IjrhZ+yy7gdwYDV6UM+qIufFuS